<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M0.532 · Pattern Recognition</p>
<p style="margin: 0; text-align:right;">Computational Engineering and Mathematics Master</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Computers, Multimedia and Telecommunications Department</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Segmentation

## import libraries

In [ ]:
!pip install opencv-contrib-python==4.4.0.44

In [ ]:
# import OpenCV library
import cv2

# we will use the following import to display images in colab:
from google.colab.patches import cv2_imshow

import numpy as np


## Load images

In [ ]:
!wget https://github.com/opencv/opencv/blob/master/samples/data/rubberwhale1.png?raw=true -O rubberwhale1.png
!wget https://github.com/opencv/opencv/blob/master/samples/data/squirrel_cls.jpg?raw=true -O squirrel_cls.jpg
!wget https://github.com/opencv/opencv/blob/master/samples/data/aero1.jpg?raw=true -O aero1.jpg

# read image
img1 = cv2.imread('rubberwhale1.png', cv2.IMREAD_COLOR)
img2 = cv2.imread('squirrel_cls.jpg', cv2.IMREAD_COLOR)
img3 = cv2.imread('aero1.jpg', cv2.IMREAD_COLOR)

# resize images to ease the display
(height, width, channels) = img1.shape
img2 = cv2.resize(img2, dsize=(width, height))
img3 = cv2.resize(img3, dsize=(width, height))

# convert image to grayscale
#img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


Display the images

In [ ]:
cv2_imshow(cv2.hconcat([img1, img2, img3]))

## GrabCut

Example from: https://docs.opencv.org/3.4/d8/d83/tutorial_py_grabcut.html

Lets work with the squirrel image:




In [ ]:
img_grabcut = img2.copy()
cv2_imshow(img_grabcut)

We need to draw a bounding box around the squirrel:

In [ ]:
# user input (position of the bounding box: initial_x, initial_y, final_x, final_y)
rect = (140,100,400,330)

# lets plot the bounding box:
color = (255,0,0)
thickness = 3
img_rect = img_grabcut.copy()
img_rect= cv2.rectangle(img_rect,(rect[0], rect[1]),(rect[2], rect[3]), color, thickness)

cv2_imshow(img_rect)

In [ ]:

mask = np.zeros(img_grabcut.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

cv2.grabCut(img_grabcut,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
result_grabcut = img_grabcut*mask2[:,:,np.newaxis]


In [ ]:
cv2_imshow(result_grabcut)

## Agglomerative Clustering


from: https://scikit-learn.org/0.15/auto_examples/cluster/plot_lena_ward_segmentation.html

In [ ]:
# import time as time
import numpy as np
# import scipy as sp
import matplotlib.pyplot as plt
from sklearn.feature_extraction.image import grid_to_graph
from sklearn.cluster import AgglomerativeClustering

# lets define a function to do the agglomerative clustering of an image:

def agglomerative_clustering_image (img, n_clusters):

    # img_reg_merg = img2.copy()
    # input image is grayscale:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ###############################################################################
    # Generate data
    X = np.reshape(img, (-1, 1))

    ###############################################################################
    # Define the structure A of the data. Pixels connected to their neighbors.
    connectivity = grid_to_graph(*img.shape)

    ###############################################################################
    # Compute clustering
    print("Compute structured hierarchical clustering...")
    ward = AgglomerativeClustering(n_clusters=n_clusters,
            linkage='ward', connectivity=connectivity).fit(X)
    label = np.reshape(ward.labels_, img.shape)
    print("Number of pixels: ", label.size)
    print("Number of clusters: ", np.unique(label).size)

    return label


def plot_aggl_clustering_contours(img, n_clusters, labels):

  ###############################################################################
  # Plot the results on an image

  plt.figure(figsize=(5, 5))
  plt.imshow(img, cmap=plt.cm.gray)

  cmap = plt.cm.get_cmap("Spectral")

  for l in range(n_clusters):
      plt.contour(labels == l,
                  colors=[cmap(l / float(n_clusters)), ])
  plt.xticks(())
  plt.yticks(())
  plt.show()


In [ ]:
n_clusters = 45  # number of regions to generate


labels_img1 = agglomerative_clustering_image (img1, n_clusters)
labels_img2 = agglomerative_clustering_image (img2, n_clusters)
labels_img3 = agglomerative_clustering_image (img3, n_clusters)

In [ ]:
plot_aggl_clustering_contours(img1, n_clusters, labels_img1)

In [ ]:
plot_aggl_clustering_contours(img2, n_clusters, labels_img2)

In [ ]:
plot_aggl_clustering_contours(img3, n_clusters, labels_img3)

## Superpixels



We will work with the SLIC superpixels. Reference example: https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_segmentations.html#sphx-glr-auto-examples-segmentation-plot-segmentations-py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from skimage.segmentation import slic

from skimage.segmentation import mark_boundaries
from skimage import img_as_ubyte

# select the input images
img1_slic = img1.copy()
img2_slic = img2.copy()
img3_slic = img3.copy()

# run the algorithm for each image
segments1_slic = slic(img1, n_segments=250, compactness=10, sigma=1,start_label=1)
segments2_slic = slic(img2, n_segments=250, compactness=10, sigma=1,start_label=1)
segments3_slic = slic(img3, n_segments=250, compactness=10, sigma=1,start_label=1)

print(f'SLIC number of segments: {len(np.unique(segments1_slic))}')
print(f'SLIC number of segments: {len(np.unique(segments2_slic))}')
print(f'SLIC number of segments: {len(np.unique(segments3_slic))}')

# draw the boundaries in the image
img1_slic = mark_boundaries(img1_slic, segments1_slic)
img2_slic = mark_boundaries(img2_slic, segments2_slic)
img3_slic = mark_boundaries(img3_slic, segments3_slic)

# go back to opencv format to display the image
cv_slic_1 = img_as_ubyte(img1_slic)
cv_slic_2 = img_as_ubyte(img2_slic)
cv_slic_3 = img_as_ubyte(img3_slic)


In [ ]:
cv2_imshow(cv2.hconcat([cv_slic_1, cv_slic_2, cv_slic_3]))


## Watersheed

Reference example:
https://docs.opencv.org/4.x/d3/db4/tutorial_py_watershed.html

In [ ]:

def watersheed_opencv(img):
  watersheed_img = img.copy()
  gray = cv2.cvtColor(watersheed_img,cv2.COLOR_BGR2GRAY)
  ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

  # noise removal
  kernel = np.ones((3,3),np.uint8)
  opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
  # sure background area
  sure_bg = cv2.dilate(opening,kernel,iterations=3)
  # Finding sure foreground area
  dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
  ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
  # Finding unknown region
  sure_fg = np.uint8(sure_fg)
  unknown = cv2.subtract(sure_bg,sure_fg)


  # Marker labelling
  ret, markers = cv2.connectedComponents(sure_fg)
  # Add one to all labels so that sure background is not 0, but 1
  markers = markers+1
  # Now, mark the region of unknown with zero
  markers[unknown==255] = 0

  markers = cv2.watershed(watersheed_img,markers)
  watersheed_img[markers == -1] = [255,255,255]
  return(watersheed_img)

In [ ]:
watersheed_img1 = watersheed_opencv(img1)
watersheed_img2 = watersheed_opencv(img2)
watersheed_img3 = watersheed_opencv(img3)

In [ ]:
cv2_imshow(cv2.hconcat([watersheed_img1, watersheed_img2, watersheed_img3]))


As an exercice, you can compare the watersheed results obtained with opencv with the ones obtained with scikit-image:

https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_segmentations.html#sphx-glr-auto-examples-segmentation-plot-segmentations-py

Can you improve the results by changing parameters?

## Graph-based segmentation: Felzenszwalb



Reference implementation: 
https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_segmentations.html#sphx-glr-auto-examples-segmentation-plot-segmentations-py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from skimage.segmentation import felzenszwalb

from skimage.segmentation import mark_boundaries
from skimage import img_as_ubyte

# select the input images
img1_felzenszwalb = img1.copy()
img2_felzenszwalb = img2.copy()
img3_felzenszwalb = img3.copy()

# run the algorithm for each image
segments1_felzenszwalb = felzenszwalb(img1_felzenszwalb, scale=100, sigma=0.5, min_size=50)
segments2_felzenszwalb = felzenszwalb(img2_felzenszwalb, scale=100, sigma=0.5, min_size=50)
segments3_felzenszwalb = felzenszwalb(img3_felzenszwalb, scale=100, sigma=0.5, min_size=50)

print(f'felzenszwalb number of segments: {len(np.unique(segments1_felzenszwalb))}')
print(f'felzenszwalb number of segments: {len(np.unique(segments2_felzenszwalb))}')
print(f'felzenszwalb number of segments: {len(np.unique(segments3_felzenszwalb))}')

# draw the boundaries in the image
img1_felzenszwalb = mark_boundaries(img1_felzenszwalb, segments1_felzenszwalb)
img2_felzenszwalb = mark_boundaries(img2_felzenszwalb, segments2_felzenszwalb)
img3_felzenszwalb = mark_boundaries(img3_felzenszwalb, segments3_felzenszwalb)

# go back to opencv format to display the image
cv_felzenszwalb_1 = img_as_ubyte(img1_felzenszwalb)
cv_felzenszwalb_2 = img_as_ubyte(img2_felzenszwalb)
cv_felzenszwalb_3 = img_as_ubyte(img3_felzenszwalb)


In [ ]:
cv2_imshow(cv2.hconcat([cv_felzenszwalb_1, cv_felzenszwalb_2, cv_felzenszwalb_3]))


# Mean Shift

Implementation using only color information (without spatial coordinates):

In [ ]:
import numpy as np    
import cv2    
from sklearn.cluster import MeanShift, estimate_bandwidth

def mean_shift_color(img_meanShift):

  # Shape of original image    
  originShape = img_meanShift.shape

  # Converting image into array of dimension [nb of pixels in originImage, 3]
  # based on r g b intensities    
  flatImg=np.reshape(img_meanShift, [-1, 3])

  # Estimate bandwidth for meanshift algorithm    
  bandwidth = estimate_bandwidth(flatImg, quantile=0.1, n_samples=100)    
  ms = MeanShift(bandwidth = bandwidth, bin_seeding=True)

  # Performing meanshift on flatImg    
  ms.fit(flatImg)

  # (r,g,b) vectors corresponding to the different clusters after meanshift    
  labels=ms.labels_

  # Remaining colors after meanshift    
  cluster_centers = ms.cluster_centers_    

  # Finding and showing the number of clusters    
  labels_unique = np.unique(labels)    
  n_clusters_ = len(labels_unique)    
  print("number of estimated clusters : %d" % n_clusters_)    

  # building final image:
  segmentedImg = cluster_centers[np.reshape(labels, originShape[:2])]

  return segmentedImg.astype(np.uint8)

In [ ]:
# executing function with the images
img_meanShift1 = mean_shift_color(img1.copy())
img_meanShift2 = mean_shift_color(img2.copy())
img_meanShift3 = mean_shift_color(img3.copy())


In [ ]:
# Displaying segmented image    
cv2_imshow(cv2.hconcat([img_meanShift1, img_meanShift2, img_meanShift3]))


Proposed exercice: create a mean shift function that also uses the spatial coordinates

# Normalized cuts

Reference code: https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_ncut.html


In [ ]:
from skimage import data, segmentation, color
from skimage.future import graph
from matplotlib import pyplot as plt

def normalized_cuts (input_image):

  # we start doing a slic segmentation. It generates the set of initial regions (400)
  labels1 = segmentation.slic(input_image, compactness=30, n_segments=400,start_label=1)

  # lets build the output image just as a reference:
  out1 = color.label2rgb(labels1, input_image, kind='avg', bg_label=0)

  # normalized cuts needs a Region Adjacency Graph (RAG): a graph showing 
  # neighborhood relations between regions
  g = graph.rag_mean_color(input_image, labels1, mode='similarity')

  # lets apply the algorithm to our graph:
  labels2 = graph.cut_normalized(labels1, g)

  # and generate the output image:
  out2 = color.label2rgb(labels2, input_image, kind='avg', bg_label=0)

  return(out1, out2)



In [ ]:
# run the algorithm with the images:
ncuts_superpixel1, ncuts_result1 = normalized_cuts(img1.copy())
ncuts_superpixel2, ncuts_result2 = normalized_cuts(img2.copy())
ncuts_superpixel3, ncuts_result3 = normalized_cuts(img3.copy())


In [ ]:
cv2_imshow(cv2.hconcat([ncuts_superpixel1, ncuts_superpixel2, ncuts_superpixel3]))


In [ ]:
cv2_imshow(cv2.hconcat([ncuts_result1, ncuts_result2, ncuts_result3]))
